# Módulo de ingesta de video: fotogramas, tiempos y métricas básicas

Este bloque de **utilidad** prepara cualquier video para análisis cuantitativo. Es la **primera etapa común** para los proyectos de:
- **Caída libre** (estimación de \(g\) a partir de \(y(t)=y_0+v_0 t+\tfrac12 g t^2\)),
- **Pelota rebotando** (energía y coeficiente de restitución),
- **Péndulo** (período \(T\), amortiguamiento),
- **Viscosímetro de esfera** (velocidad terminal \(v_t\) y \(\mu\)),
- y en general, **cualquier experimento** que parta de un video y necesite un *pipeline* reproducible.

## ¿Qué hace exactamente?

1. **Abre** un archivo de video y verifica acceso.
2. **Lee todos los fotogramas** y extrae el **timestamp real** de cada uno (en segundos) usando `CAP_PROP_POS_MSEC`  
   → robusto incluso si el teléfono grabó con **FPS variable**.
3. **Calcula métricas**:  
   - **Duración total** (según lectura real),  
   - **FPS promedio** (frames leídos / duración),  
   - **# de fotogramas**,  
   - y además lee los **metadatos** de contenedor (FPS y #frames reportados).
4. **Exporta fotogramas** (opcional) a una carpeta (`frames/`) y  
   **genera un CSV** `frames_index.csv` con el mapeo:

In [1]:
# Requisitos: pip install opencv-python-headless numpy pandas

from pathlib import Path
import cv2, numpy as np, pandas as pd

def video_report(video_path, out_dir="frames", every=1, save_frames=True):
    video_path = Path(video_path)
    out_dir = Path(out_dir)
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        raise RuntimeError(f"No se pudo abrir el video: {video_path}")

    # Metadatos del contenedor (pueden ser inexactos)
    meta_fps = cap.get(cv2.CAP_PROP_FPS) or 0.0
    meta_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
    meta_duration = (meta_frames / meta_fps) if (meta_fps > 0 and meta_frames > 0) else None

    if save_frames:
        out_dir.mkdir(parents=True, exist_ok=True)

    timestamps = []
    frames_read = 0
    frames_saved = 0

    while True:
        ok, frame = cap.read()
        if not ok:
            break
        # tiempo real del frame (soporta FPS variable)
        t = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0
        timestamps.append(t)

        if save_frames and (every == 1 or frames_read % every == 0):
            cv2.imwrite(str(out_dir / f"frame_{frames_read:06d}.jpg"), frame)
            frames_saved += 1

        frames_read += 1

    cap.release()

    if timestamps:
        duration_read = timestamps[-1]
        fps_mean = frames_read / duration_read if duration_read > 0 else float("nan")
        df = pd.DataFrame({"frame_idx": np.arange(frames_read), "t_seconds": timestamps})
        if save_frames:
            df.to_csv(out_dir / "frames_index.csv", index=False)
    else:
        duration_read = 0.0
        fps_mean = float("nan")

    # Reporte
    print("\n== INFORME DEL VIDEO ==")
    print(f"Archivo:            {video_path}")
    print(f"Frames leídos:      {frames_read}")
    print(f"Duración leída:     {duration_read:.3f} s")
    print(f"FPS promedio leído: {fps_mean:.3f}" if np.isfinite(fps_mean) else "FPS promedio leído: n/a")
    print("\n-- Metadatos del contenedor --")
    print(f"FPS (metadatos):    {meta_fps:.3f}" if meta_fps > 0 else "FPS (metadatos): n/d")
    print(f"#Frames (metadatos): {meta_frames}" if meta_frames > 0 else "#Frames (metadatos): n/d")
    if meta_duration is not None:
        print(f"Duración (metadatos): {meta_duration:.3f} s")
    if save_frames:
        print(f"\nFrames guardados:   {frames_saved}  en {out_dir.resolve()}")

    if meta_fps > 0 and np.isfinite(fps_mean) and abs(fps_mean - meta_fps) / meta_fps > 0.05:
        print("\nAviso: diferencia >5% entre FPS de metadatos y promedio leído (posible FPS variable).")

    return {
        "frames_read": frames_read,
        "duration_read": duration_read,
        "fps_mean": fps_mean,
        "meta_fps": meta_fps,
        "meta_frames": meta_frames,
        "meta_duration": meta_duration,
        "frames_saved": frames_saved,
        "out_dir": str(out_dir.resolve()),
    }

In [2]:
# Ejemplo de uso en notebook:
res = video_report("Video2025-08-18.mp4", out_dir="frames", every=2, save_frames=True)


== INFORME DEL VIDEO ==
Archivo:            Video2025-08-18.mp4
Frames leídos:      80
Duración leída:     2.633 s
FPS promedio leído: 30.380

-- Metadatos del contenedor --
FPS (metadatos):    30.000
#Frames (metadatos): 80
Duración (metadatos): 2.667 s

Frames guardados:   40  en /Users/neno/Documents/Cursos/SensadoModelado/frames


In [3]:
print("Cada frame dura:", 2.633/80, "segundos")

Cada frame dura: 0.0329125 segundos
